<a href="https://colab.research.google.com/github/shahdhruv29/Big-Data-Processing-Labs/blob/main/labs/Lab4_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 04 - Programming Spark SQL**

# **Dhruv Shah - 202103017**

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=c322569e0d6e4e26d3a03fbf1459897f711704692c3993f8a6cf61c680c324a8
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc, col
spark = SparkSession.builder.appName("US-Sales").getOrCreate()

In [ ]:
csv_folder_path = "/content/gdrive/My Drive/datasets/us-sales/orders.csv"

# Read all CSV files in the folder into a DataFrame
df_orders = spark.read.option("header", "true").csv(csv_folder_path)
df_stores = spark.read.option("header", "true").csv("/content/gdrive/My Drive/datasets/us-sales/stores.csv")
df_customers = spark.read.option("header", "true").csv("/content/gdrive/My Drive/datasets/us-sales/customers.csv")
df_products = spark.read.option("header", "true").csv("/content/gdrive/My Drive/datasets/us-sales/products.csv")
df_regions = spark.read.option("header", "true").csv("/content/gdrive/My Drive/datasets/us-sales/regions.csv")
df_sales_teams = spark.read.option("header", "true").csv("/content/gdrive/My Drive/datasets/us-sales/sales_teams.csv")

Q1. Compute top-10 selling products in terms of numbers (i. e. sum(qty))

In [ ]:
from pyspark.sql.functions import sum, desc
top_selling_products = df_orders.groupBy("ProductID").agg(sum("OrderQuantity").alias("TotalQuantity")).orderBy(desc("TotalQuantity")).limit(10)

print("Top 10 Selling Products (by Total Quantity Sold):")
top_selling_products.show()

Top 10 Selling Products (by Total Quantity Sold):
+---------+-------------+
|ProductID|TotalQuantity|
+---------+-------------+
|       23|        956.0|
|       37|        896.0|
|        8|        879.0|
|        4|        878.0|
|       40|        855.0|
|       41|        854.0|
|       22|        837.0|
|       38|        832.0|
|       27|        830.0|
|       12|        827.0|
+---------+-------------+



Q2. Compute top-10 selling products in terms of value (i. e. sum (qty*price))

In [ ]:
top_selling_products_by_value = df_orders.groupBy("ProductID").agg(sum(df_orders["OrderQuantity"] * df_orders["UnitPrice"]).alias("TotalValue")) \
    .orderBy(desc("TotalValue")).limit(10)

print("Top 10 Selling Products (by Total Value Sold):")
top_selling_products_by_value.show()

Top 10 Selling Products (by Total Value Sold):
+---------+------------------+
|ProductID|        TotalValue|
+---------+------------------+
|       23| 2358788.599999999|
|       40|         2130841.2|
|        4|2071546.1999999993|
|       37|         2052886.7|
|       41|         2049958.8|
|        5|2011333.3000000012|
|        2|         2005638.3|
|       35| 1981973.900000001|
|        8|1976895.2999999996|
|       17|1925111.0000000002|
+---------+------------------+



Q3. Compute top-10 profit making products. Profit = sum(qty*(price-cost))

In [ ]:
top_profit_making_products = df_orders.groupBy("ProductID").agg(sum(df_orders["OrderQuantity"] * (df_orders["UnitPrice"] - df_orders["UnitCost"])).alias("TotalProfit")) \
    .orderBy(desc("TotalProfit")).limit(10)

print("Top 10 Profit-Making Products:")
top_profit_making_products.show()

Top 10 Profit-Making Products:
+---------+-----------------+
|ProductID|      TotalProfit|
+---------+-----------------+
|       23|908818.7699999997|
|        8|796037.5299999998|
|        4|786277.2900000003|
|        2|783599.7399999995|
|       40|767278.9100000005|
|       41|761318.8800000004|
|        5|750981.4400000005|
|       37|743189.7299999997|
|       35|741447.8800000004|
|       11|741098.0599999997|
+---------+-----------------+



Q4. Give top-3 stores selling product product number 25

In [ ]:
# Filter orders for product number 25
product_25_orders = df_orders.filter(df_orders["ProductID"] == 25)

# Group by store and sum the quantities sold
top_stores_product_25 = product_25_orders.groupBy("StoreID").agg(sum("OrderQuantity").alias("TotalQuantity")).orderBy(desc("TotalQuantity")).limit(3)

print("Top 3 Stores Selling Product 25:")
top_stores_product_25.show()

Top 3 Stores Selling Product 25:
+-------+-------------+
|StoreID|TotalQuantity|
+-------+-------------+
|     56|         16.0|
|     26|         14.0|
|    350|         13.0|
+-------+-------------+



Q5. Give top-3 products sold in midwest region

In [ ]:
table1 = df_orders.join(df_products, df_orders["ProductID"] == df_products["ProductID"]).drop(df_orders["ProductID"])
table2 = table1.join(df_stores, table1["StoreID"] == df_stores["_StoreID"]).drop(df_stores["_StoreID"])
table3 = table2.join(df_regions, table2["StateCode"] == df_regions["StateCode"]).drop(df_regions["StateCode"]).where(col("Region") == "Midwest")

# Group by Region, ProductID, and ProductName and sum the quantities sold
top_products_midwest = table3.groupBy("Region", "ProductID", "ProductName").agg(sum("OrderQuantity").alias("Total Quantity")).orderBy(desc("Total Quantity")).limit(3)

print("Top 3 Products Sold in Midwest Region:")
top_products_midwest.show()

Top 3 Products Sold in Midwest Region:
+-------+---------+------------+--------------+
| Region|ProductID| ProductName|Total Quantity|
+-------+---------+------------+--------------+
|Midwest|       25|TV and video|         224.0|
|Midwest|       29|    Pendants|         206.0|
|Midwest|       23| Accessories|         206.0|
+-------+---------+------------+--------------+



Q6. Give region wise quantity sold for each product. Compute: Region, Product ID, Sum(Qty). Region is related to a order through sales team.

In [ ]:
# Join df_orders with df_stores and df_sales_teams
table1 = df_orders.join(df_stores, df_orders["StoreID"] == df_stores["_StoreID"]).join(df_sales_teams, "SalesTeamID", "inner")

# Group by region and product, and sum the quantities sold
region_product_quantity = table1.groupBy("Region", "ProductID").agg(sum("OrderQuantity").alias("TotalQuantity"))

print("Region-wise Quantity Sold for Each Product:")
region_product_quantity.show()


Region-wise Quantity Sold for Each Product:
+---------+---------+-------------+
|   Region|ProductID|TotalQuantity|
+---------+---------+-------------+
|    South|       10|        203.0|
|  Midwest|       20|        224.0|
|     West|       15|        192.0|
|    South|       18|        144.0|
|  Midwest|       28|        209.0|
|     West|        9|        262.0|
|  Midwest|       45|        264.0|
|  Midwest|        1|        192.0|
|  Midwest|        7|        231.0|
|     West|        1|        158.0|
|    South|        6|         71.0|
|  Midwest|       25|        246.0|
|Northeast|       42|        162.0|
|    South|       40|        165.0|
|     West|       13|        259.0|
|    South|       44|        127.0|
|  Midwest|       36|        179.0|
|Northeast|       33|        190.0|
|  Midwest|       22|        247.0|
|  Midwest|        2|        212.0|
+---------+---------+-------------+
only showing top 20 rows



Q7. Compute Average monthly sale in terms of numbers at each store; that , that is on average what numbers of a product are sold on a store in a month.

In [ ]:
from pyspark.sql.functions import year, month,avg

# Convert OrderDate to a date type
df1 = df_orders.withColumn("OrderDate", df_orders["OrderDate"].cast("date"))

# Extract year and month from OrderDate
df1 = df_orders.withColumn("Year", year("OrderDate")).withColumn("Month", month("OrderDate"))

# Group by store, year, and month, and compute the average quantity sold
average_monthly_sale = df1.groupBy("StoreID", "Year", "Month").agg(avg("OrderQuantity").alias("AverageQuantity"))

print("Average Monthly Sale at Each Store:")
average_monthly_sale.show()

Average Monthly Sale at Each Store:
+-------+----+-----+-----------------+
|StoreID|Year|Month|  AverageQuantity|
+-------+----+-----+-----------------+
|    231|2018|    6|              3.5|
|    358|2018|    7|              7.0|
|    360|2018|    7|              5.0|
|    286|2018|    9|              5.0|
|    330|2018|   10|              7.0|
|    262|2018|   11|              2.0|
|    147|2019|    2|              4.0|
|    172|2019|    2|              6.0|
|    202|2019|    3|              2.0|
|     72|2019|    4|              7.0|
|     84|2019|    4|              3.0|
|    320|2019|    5|              4.0|
|    151|2019|    6|              1.0|
|    344|2019|    6|              1.0|
|    161|2019|    8|              2.0|
|     60|2019|    8|              7.0|
|     38|2019|   11|              3.0|
|     41|2019|   12|              3.5|
|     30|2020|    1|              6.0|
|     84|2020|    2|4.333333333333333|
+-------+----+-----+-----------------+
only showing top 20 rows



Q8. Compute sales bifurcation of each warehouse; that total sales amount through each channel

In [ ]:
from pyspark.sql.functions import sum, col

# Group by WarehouseCode and Sales_Channel and calculate the sales amount
sales_bifurcation = df_orders.groupBy("WarehouseCode", "Sales_Channel") \
    .agg(sum(col("OrderQuantity") * col("UnitPrice")).alias("Sales Amount")) \
    .orderBy(col("Sales Amount"))

print("Sales Bifurcation of Each Warehouse:")
sales_bifurcation.show()

Sales Bifurcation of Each Warehouse:
+-------------+-------------+------------------+
|WarehouseCode|Sales_Channel|      Sales Amount|
+-------------+-------------+------------------+
| WARE-NBV1002|    Wholesale| 703781.3999999998|
| WARE-MKL1006|    Wholesale|          837071.2|
| WARE-NBV1002|  Distributor|1248377.5000000002|
| WARE-XYS1001|    Wholesale|1301287.4000000006|
| WARE-UHY1004|    Wholesale|1501536.9999999998|
| WARE-PUJ1005|    Wholesale|1713940.4000000004|
| WARE-MKL1006|  Distributor|1738784.0000000002|
| WARE-XYS1001|  Distributor|2099110.0000000005|
| WARE-NBV1002|       Online| 2103324.299999999|
| WARE-UHY1004|  Distributor|         2269785.8|
| WARE-MKL1006|       Online|         2606648.4|
| WARE-NBV1002|     In-Store|2856270.3000000003|
| WARE-PUJ1005|  Distributor|2998256.6999999993|
| WARE-NMK1003|    Wholesale| 3155331.499999998|
| WARE-MKL1006|     In-Store|3554832.4000000004|
| WARE-XYS1001|       Online| 3827207.500000005|
| WARE-UHY1004|       Online| 40

Q9. Compute average "product retention period" (i. e. the difference between procurement date and order date) at each warehouse

In [ ]:
# Check how many rows have NULL values in OrderDate or ProcuredDate
null_rows_count = df_orders.filter(col("OrderDate").isNull() | col("ProcuredDate").isNull()).count()
print(f"Number of rows with NULL values in OrderDate or ProcuredDate: {null_rows_count}")

# Show a few examples of the rows with NULL values
df_orders.filter(col("OrderDate").isNull() | col("ProcuredDate").isNull()).show()


Number of rows with NULL values in OrderDate or ProcuredDate: 7991
+-----------+-------------+-------------+------------+----------+----------+------------+------------+-----------+----------+-------+---------+-------------+---------------+---------+--------+
|OrderNumber|Sales_Channel|WarehouseCode|ProcuredDate| OrderDate|  ShipDate|DeliveryDate|CurrencyCode|SalesTeamID|CustomerID|StoreID|ProductID|OrderQuantity|DiscountApplied|UnitPrice|UnitCost|
+-----------+-------------+-------------+------------+----------+----------+------------+------------+-----------+----------+-------+---------+-------------+---------------+---------+--------+
|SO - 000101|     In-Store| WARE-UHY1004|        NULL|2018-05-31|14-06-2018|  19-06-2018|         USD|          6|        15|    259|       12|            5|          0.075|   1963.1| 1001.18|
|SO - 000102|       Online| WARE-NMK1003|        NULL|2018-05-31|22-06-2018|  02-07-2018|         USD|         14|        20|    196|       27|            3|    

In [ ]:
# from pyspark.sql.functions import datediff, avg

# # Calculate the difference between OrderDate and ProcuredDate
# date_difference = df_orders.select(
#     col("WarehouseCode"),
#     datediff(col("OrderDate"), col("ProcuredDate")).alias("Date_difference")
# )

# # Group by WarehouseCode and calculate the average product retention period
# average_retention_period = date_difference.groupBy("WarehouseCode") \
#     .agg(avg('Date_difference').alias("product retention period")) \
#     .orderBy(col("product retention period").asc())

# print("Average Product Retention Period at Each Warehouse:")
# average_retention_period.show()

from pyspark.sql.functions import col, date_format, substring, to_date


date_difference = df_orders.select(
    col("WarehouseCode"),
    ((date_format("OrderDate", "yyyy").cast("int") * 365) +
     (date_format("OrderDate", "MM").cast("int") * 12) +
     (date_format("OrderDate", "dd").cast("int")) -
     (substring("ProcuredDate", -4, 4).cast("int") * 365) -
     (substring("ProcuredDate", -7, 2).cast("int") * 12) -
     (substring("ProcuredDate", 0, 2).cast("int"))
    ).alias("avg_diff_in_days")
)

# Group by WarehouseCode and calculate the average product retention period
average_retention_period = date_difference.groupBy("WarehouseCode") \
    .agg(avg("avg_diff_in_days").alias("product_retention_period")) \
    .orderBy(col("product_retention_period").asc())

print("Average Product Retention Period at Each Warehouse:")
average_retention_period.show()


Average Product Retention Period at Each Warehouse:
+-------------+------------------------+
|WarehouseCode|product_retention_period|
+-------------+------------------------+
| WARE-XYS1001|                    NULL|
| WARE-PUJ1005|                    NULL|
| WARE-MKL1006|                    NULL|
| WARE-NMK1003|                    NULL|
| WARE-UHY1004|                    NULL|
| WARE-NBV1002|                    NULL|
+-------------+------------------------+



Q10. Give Year-Month sales of all products. Here you actually print 'Year-Month', ProductID, sum(qty). Use Order Date for extracting Year and Month of sale. For simplicity you can read order date as string only in YYYY-MM-DD format, and extract the required info accordingly.

In [ ]:
from pyspark.sql.functions import year, month, sum

# Extract year and month from OrderDate
year_month_sale = df_orders.withColumn("Year", year("OrderDate")).withColumn("Month", month("OrderDate"))

# Group by Year, Month, and ProductID, and calculate the total quantity sold
year_month_sale = year_month_sale.groupBy("Year", "Month", "ProductID") \
    .agg(sum(col("OrderQuantity")).alias("Total Quantity"))

print("Year-Month Sale of All Products:")
year_month_sale.show()

Year-Month Sale of All Products:
+----+-----+---------+--------------+
|Year|Month|ProductID|Total Quantity|
+----+-----+---------+--------------+
|2018|    8|       45|          11.0|
|2018|   12|        1|          30.0|
|2019|    5|       40|          53.0|
|2019|   10|        4|          12.0|
|2020|    1|       10|          28.0|
|2018|   10|       25|          33.0|
|2019|    3|       30|          24.0|
|2020|    6|       28|           9.0|
|2020|    6|       32|          34.0|
|2019|   12|       16|          22.0|
|2020|   11|       43|          39.0|
|2019|    1|       23|          22.0|
|2019|    9|       42|          23.0|
|2020|    9|        5|          18.0|
|2018|   10|       13|          12.0|
|2019|   10|       19|          41.0|
|2018|   12|        3|          12.0|
|2019|    3|        1|          19.0|
|2019|    3|        4|          18.0|
|2019|    4|       33|          38.0|
+----+-----+---------+--------------+
only showing top 20 rows



Q11. Compute a fact file with the dimensions of "store_id", "product_id", and "month_year". Let facts to be computed are "quantity" and "amount". Let month_year be represented as YYYY-MM.

In [ ]:
# Calculate quantity and amount for each combination of StoreID, ProductID, and OrderDate
fact_file = df_orders.cube("StoreID", "ProductID", "OrderDate") \
    .agg(sum(col("OrderQuantity")).alias("Quantity"), sum(col("OrderQuantity") * col("UnitPrice")).alias("Amount")) \
    .sort("StoreID", "ProductID", "OrderDate")

print("Fact File with Dimensions of StoreID, ProductID, and Month-Year (YYYY-MM):")
fact_file.show()


Fact File with Dimensions of StoreID, ProductID, and Month-Year (YYYY-MM):
+-------+---------+----------+--------+-------------------+
|StoreID|ProductID| OrderDate|Quantity|             Amount|
+-------+---------+----------+--------+-------------------+
|   NULL|     NULL|      NULL| 36162.0|8.269272660000001E7|
|   NULL|     NULL|2018-05-31|    39.0|  75629.59999999999|
|   NULL|     NULL|2018-06-01|    62.0|           148961.1|
|   NULL|     NULL|2018-06-02|    35.0|            79696.5|
|   NULL|     NULL|2018-06-03|    59.0| 214125.30000000002|
|   NULL|     NULL|2018-06-04|    26.0| 100392.80000000002|
|   NULL|     NULL|2018-06-05|    32.0|  89203.79999999999|
|   NULL|     NULL|2018-06-06|    20.0|            45332.2|
|   NULL|     NULL|2018-06-07|    40.0| 112593.50000000001|
|   NULL|     NULL|2018-06-08|    36.0| 39014.100000000006|
|   NULL|     NULL|2018-06-09|    37.0|  64420.50000000001|
|   NULL|     NULL|2018-06-10|    19.0| 52414.100000000006|
|   NULL|     NULL|2018-0